In [1]:
import serial
import time
import numpy as np
import serial.tools.list_ports
import matplotlib.pyplot as plt
port_list = [port.device for port in serial.tools.list_ports.comports()]
print(port_list)

['COM8', 'COM3']


In [19]:
port.close()

In [20]:
# Pick a port from port list:
port = serial.Serial(port_list[0],115200)

In [21]:
def send_msg(msg):
    try:
        byte_msg = (msg + '\n').encode('utf-8')
        port.write(byte_msg)
        print('message sent: ' + msg)
    except:
        print('message invalid')

In [7]:
def set_freq(freq):
    send_msg('wavegen set frequency '+str(freq))
    send_msg('wavegen start')

In [8]:
# Test serial port. Lights should animate for 3 seconds (skips lights connected to sensors)
send_msg('rgb start')
time.sleep(2)
send_msg('rgb stop')

message sent: rgb start
message sent: rgb stop


In [24]:
# initialize VibeCheck with desired data rate, amplitude
send_msg('sensor 0 set accel odr 200')
send_msg('sensor 0 set accel range 8')
send_msg('wavegen set amplitude 0.5')

message sent: sensor 0 set accel odr 200
message sent: sensor 0 set accel range 8
message sent: wavegen set amplitude 0.5


In [10]:
def parse_data(raw_data):
    num_points = int( raw_data[1] )
    data_point_arr = [] 
    for i in range(num_points):
        # 5 is number of elements in each datapoint, 2 is index of first piece of data
        start_index = i * 5 + 2
        data_point = raw_data[start_index:start_index+5]
        data_point_arr.append(data_point)
    return data_point_arr

In [222]:
# Create a list of discrete frequencies for the independent axis
freq_list = np.linspace(20, 50, 10) # frequency: start, stop, num
num_points = 1000 # number of datapoints to record at each frequency

for freq in freq_list:
    send_msg('wavegen start')
    send_msg('wavegen set frequency '+str(freq))
    time.sleep(2)

message sent: wavegen start
message sent: wavegen set frequency 20.0
message sent: wavegen start
message sent: wavegen set frequency 23.333333333333332
message sent: wavegen start
message sent: wavegen set frequency 26.666666666666668
message sent: wavegen start
message sent: wavegen set frequency 30.0
message sent: wavegen start
message sent: wavegen set frequency 33.333333333333336
message sent: wavegen start
message sent: wavegen set frequency 36.66666666666667
message sent: wavegen start
message sent: wavegen set frequency 40.0
message sent: wavegen start
message sent: wavegen set frequency 43.333333333333336
message sent: wavegen start
message sent: wavegen set frequency 46.66666666666667
message sent: wavegen start
message sent: wavegen set frequency 50.0


In [27]:
def record_samples(num_samples, settle_time):
    samples_recorded = 0
    sample_arr = np.zeros([num_samples, 5]) # each datapoint will be: [channel,t,x,y,z]
    send_msg('sensor 0 start accel')
    time.sleep(settle_time)
    print('done sleeping')
    port.flush() # make sure nothing is in serial input buffer 
    print("begin sampling")
    port.readline()
    
    while samples_recorded < num_samples:
        if port.in_waiting > 0: # if there is data waiting in the serial input buffer
            rawline = port.readline()
            print(rawline)
            newline = rawline.decode().strip().split(" ")
            #print(newline)
            if newline[0] == "data": # if received packet is a data packet
                data = parse_data(newline)
                #print(data)
                for dp in data:
                    if(len(dp) == 0 or dp[0] != '0'):
                        print('bad datapoint')
                    else:
                        sample_arr[samples_recorded] = dp
                        samples_recorded += 1
                    if samples_recorded > num_samples:
                        break
                
            else:
                pass
                #print(newline)

    send_msg('sensor 0 stop accel')
    print(str(num_samples) + ' samples recorded')
    return sample_arr

In [28]:
## for testing ##

send_msg('wavegen set frequency 60')
send_msg('wavegen start')
frequency_sample = record_samples(400, 2)
send_msg('wavegen stop')
frequency_sample[frequency_sample[:,1].argsort()] # sort datapoints by time to make sure none are out of order
t = frequency_sample[:,1]
z= frequency_sample[:,4]
plt.plot(t,z)

message sent: wavegen set frequency 60
message sent: wavegen start
message sent: sensor 0 start accel
done sleeping
begin sampling
b'data 10 0 119258189 0.700928 0.710693 0.031982 0 119262831 0.696533 0.707031 0.032471 0 119267473 0.691406 0.698730 0.031738 0 119272116 0.688965 0.694580 0.031250 0 119276758 0.687256 0.690674 0.031250 0 119281400 0.683105 0.687012 0.029053 0 119286041 0.684570 0.683838 0.030762 0 119290684 0.681641 0.683594 0.031250 0 119295326 0.683594 0.683594 0.029297 0 119299968 0.681152 0.683838 0.030518\n'
b'data 10 0 119304610 0.684570 0.686035 0.028564 0 119309253 0.686279 0.688965 0.031494 0 119313895 0.689697 0.691895 2383 0.698975 0.031494 0 119327824 0.696045 0.701904 0.030273 0 119332466 0.699707 0.706299 0.029297 0 119337108 0.701416 0.709473 0.031250 0 119341750 0.702148 0.710205 0.030762 0 119346391 0.703857 0.712891 0.031738\n'
bad datapoint
bad datapoint
bad datapoint
bad datapoint
bad datapoint
bad datapoint
bad datapoint
b'data 10 0 119351034 0.70507

IndexError: index 400 is out of bounds for axis 0 with size 400

In [206]:
send_msg('sensor 0 stop accel')

message sent: sensor 0 stop accel


SyntaxError: incomplete input (705908127.py, line 2)